# Untruncated Output Logging in Jupyter
This notebook configures robust logging that captures full, untruncated outputs to a log file and lets you view them live from a terminal.

In [11]:
# 1) Configure Paths and Unbuffered Streams
import os, sys, time
from pathlib import Path
from datetime import datetime

# Logs directory and timestamped filename
LOGS_DIR = Path('logs')
LOGS_DIR.mkdir(parents=True, exist_ok=True)
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
LOG_PATH = LOGS_DIR / f'jupyter_untruncated_{ts}.log'

# Reduce buffering for interactive logging
try:
    sys.stdout.reconfigure(line_buffering=True)
except Exception:
    pass
try:
    sys.stderr.reconfigure(line_buffering=True)
except Exception:
    pass

# Ensure child processes are unbuffered
os.environ['PYTHONUNBUFFERED'] = '1'
print(f'Log file will be written to: {LOG_PATH}')

In [12]:
# 2) Initialize Logging (Console + Rotating File)
import logging
from logging.handlers import RotatingFileHandler

# Reset logging in notebooks and attach both console and rotating file handlers
logger = logging.getLogger()
for h in list(logger.handlers):
    logger.removeHandler(h)

log_formatter = logging.Formatter(
    fmt='%(asctime)s | %(levelname)-8s | %(name)s | %(module)s:%(lineno)d | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
 )

console_handler = logging.StreamHandler(sys.stderr)
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(log_formatter)

file_handler = RotatingFileHandler(LOG_PATH, maxBytes=50*1024*1024, backupCount=3, encoding='utf-8')
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(log_formatter)

logging.basicConfig(level=logging.DEBUG, handlers=[console_handler, file_handler], force=True)
logger = logging.getLogger('untruncated')
logger.info('Logging initialized; streaming to console and %s', LOG_PATH)

In [13]:
# 3) Tee sys.stdout and sys.stderr to the Log
import io

# Preserve originals
ORIG_STDOUT = sys.stdout
ORIG_STDERR = sys.stderr

# Open a dedicated append-only, line-buffered stream to the same log file
log_fp = open(LOG_PATH, 'a', encoding='utf-8', buffering=1)  # line-buffered

class Tee(io.TextIOBase):
    def __init__(self, primary, secondary):
        self.primary = primary
        self.secondary = secondary
    def write(self, s):
        try:
            self.primary.write(s)
        except Exception:
            pass
        try:
            self.secondary.write(s)
        except Exception:
            pass
        self.flush()
        return len(s)
    def flush(self):
        try:
            self.primary.flush()
        except Exception:
            pass
        try:
            self.secondary.flush()
        except Exception:
            pass
    def isatty(self):
        try:
            return self.primary.isatty()
        except Exception:
            return False
    def fileno(self):
        # Provide fileno when available on primary; otherwise raise
        if hasattr(self.primary, 'fileno'):
            try:
                return self.primary.fileno()
            except Exception:
                pass
        raise OSError('fileno not available')

sys.stdout = Tee(ORIG_STDOUT, log_fp)
sys.stderr = Tee(ORIG_STDERR, log_fp)
print('Tee installed; stdout/stderr will be written to both console and log file.')
logger.info('Tee active. All prints and errors will be mirrored to %s', LOG_PATH)

In [14]:
# 4) Generate Large Output (Stress Test)
import os, math
logger.info('Starting large output stress test...')
print('Beginning large output...')
for i in range(5000):  # adjust higher if desired
    # Mixed print/logging to create substantial output
    print(f'Line {i:05d} - print spam ' + ('x'*100))
    if i % 100 == 0:
        logger.debug('Checkpoint %d reached with PI≈%.6f', i, math.pi)
    if i % 500 == 0:
        # Force flushes to disk periodically
        try:
            log_fp.flush()
            os.fsync(log_fp.fileno())
        except Exception:
            pass
print('Large output complete.')
logger.info('Large output stress test complete.')

In [15]:
# 5) Disable Common Library Truncation (pandas, numpy)
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold=np.inf, linewidth=200)

# Quick demo: large DataFrame to verify settings
df_demo = pd.DataFrame({'a': range(200), 'b': ['x'*20]*200})
print('DataFrame demo (first 10 rows shown inline by code, full in log if printed in one go):')
print(df_demo.to_string(index=False))

In [16]:
# 6) Stream Log Updates in-Notebook (optional)
import threading, time
from collections import deque

TAILER_RUNNING = False
TAILER_THREAD = None

def tail_log(path, n_last=20, interval=0.5):
    global TAILER_RUNNING
    TAILER_RUNNING = True
    try:
        with open(path, 'r', encoding='utf-8') as f:
            # Seek to end
            f.seek(0, os.SEEK_END)
            while TAILER_RUNNING:
                lines = f.readlines()
                if lines:
                    # Keep only last n_last lines in output
                    dq = deque(lines, maxlen=n_last)
                    print(''.join(dq), end='')
                time.sleep(interval)
    except Exception as e:
        print(f'Tailer encountered an error: {e}', file=sys.stderr)

def start_tailer():
    global TAILER_THREAD
    if TAILER_THREAD and TAILER_THREAD.is_alive():
        print('Tailer already running.')
        return
    TAILER_THREAD = threading.Thread(target=tail_log, args=(LOG_PATH,), daemon=True)
    TAILER_THREAD.start()
    print('Tailer started.')

def stop_tailer():
    global TAILER_RUNNING, TAILER_THREAD
    TAILER_RUNNING = False
    if TAILER_THREAD:
        TAILER_THREAD.join(timeout=2)
        print('Tailer stopped.')

# Start tailing if you want to see live updates in the cell output
# start_tailer()

In [17]:
# 7) Emit a VS Code Terminal Tail Command
cmd = f"tail -f '{LOG_PATH}'"
print('To follow logs live in the VS Code terminal, run:')
print(cmd)

In [18]:
# 8) Capture Exceptions with Full Tracebacks
import traceback, asyncio

def log_unhandled_exception(exc_type, exc_value, exc_tb):
    logger = logging.getLogger('untruncated')
    logger.exception('Uncaught exception', exc_info=(exc_type, exc_value, exc_tb))
    # Also print to ensure it goes through Tee
    print('Uncaught exception:', ''.join(traceback.format_exception(exc_type, exc_value, exc_tb)))

sys.excepthook = log_unhandled_exception

try:
    loop = asyncio.get_event_loop()
    def handle_async_exception(loop, context):
        msg = context.get('exception') or context.get('message')
        logging.getLogger('untruncated').error('Async exception: %s', msg, exc_info=context.get('exception'))
    loop.set_exception_handler(handle_async_exception)
except Exception:
    pass
print('Exception hooks installed.')

In [19]:
# 9) Restore Original Streams (Cleanup)
def cleanup_logging():
    # Stop tailer if running
    try:
        stop_tailer()
    except Exception:
        pass
    # Restore original streams
    try:
        sys.stdout = ORIG_STDOUT
        sys.stderr = ORIG_STDERR
    except Exception:
        pass
    # Close log file pointer
    try:
        log_fp.flush()
        log_fp.close()
    except Exception:
        pass
    # Remove logging handlers
    root = logging.getLogger()
    for h in list(root.handlers):
        try:
            root.removeHandler(h)
            h.flush()
            h.close()
        except Exception:
            pass
    print('Logging cleaned up. Subsequent prints will not be teed.')

# Run this when you want to stop teeing and close files
# cleanup_logging()